In [1]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
#loading the data
# Load data from a text file
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            label, sentence = line.strip().split('\t', 1)  # Split on the first tab character
            data.append([sentence, int(label)])
    return pd.DataFrame(data, columns=['sentence', 'label'])

# file_path
file_path = 'TRAINING_DATA.txt'

#assign the data into pandas DF
data_2 = load_data(file_path)

data_2.head()

,sentence,label
0,"Cuando conocí a Janice en 2013 , una familia n...",1
1,Hwang habló en Sur de este año por Southwest M...,0
2,Usted podría pensar Katy Perry y Robert Pattin...,1
3,Cualquiera que haya volado los cielos del crea...,1
4,"Bueno , este cantante tendrá un LARGO tiempo p...",1


In [3]:
sampled_df = data_2.sample(frac=0.3, random_state=42)

In [4]:

# Load pre-trained Spanish BETO model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the embedding for the [CLS] token
    embedding = outputs.last_hidden_state[0][0].numpy()
    return embedding

# Apply embedding to the 'sentences' column
sampled_df['embedding'] = sampled_df['sentence'].apply(embed_text)

# Split the data
X = np.array(sampled_df['embedding'].tolist())
y = sampled_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the model
gbc = GradientBoostingClassifier()

# Initialize Grid Search
grid_search = GridSearchCV(estimator=gbc, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fitting 3 folds for each of 1458 candidates, totalling 4374 fits


KeyboardInterrupt: 